ライブラリのimport

In [5]:
import keras
from keras.models import Model
from keras.layers import Input, Dense, Activation, Dropout
from keras.layers import Conv2D, GlobalAveragePooling2D
from keras.layers import BatchNormalization, Add
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from keras.initializers import he_normal
import keras.backend as K
import sqlalchemy as sa
import pandas as pd
import data.constant as con
from tqdm import tqdm
import component.create_raceID as cr
import classes.db_operation_class as db
import classes.convert_df as convert
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import component.create_raceID as cr
from tqdm import tqdm
import pickle



In [6]:
# インスタンスの作成
db_instans = db.Main()
convert_instans = convert.Main()
raceIdList = cr.get_save_race_id()

# カラのdf生成
cols = ['RANKING','HORSEFRAME','HORSENUMBER','WEIGHT','HORSE_WEIGHT','WEIGHT_GAIN_LOSS','ODDS','POPULAR']
# cols = ['RANKING','HORSEFRAME','HORSENUMBER','WEIGHT','HORSE_WEIGHT','WEIGHT_GAIN_LOSS']

# df = pd.DataFrame(columns=cols)
df = convert_instans.convert_df("RESULT_HORSE","202301010101")

print(df)

# # raceIdList分のレースを結合する
# for race_id in tqdm(raceIdList):
#   df1 = convert_instans.convert_df("RESULT_HORSE",race_id)

#   df = pd.concat([df, df1], ignore_index=True)

       RANKING       RACE_ID  HORSEFRAME  HORSENUMBER      HNAME    HORSE_ID  \
0            1  201901010101           1            1      ゴルコンダ  2017105318   
1            2  201901010101           3            3   プントファイヤー  2017104612   
2            3  201901010101           4            4  ラグリマスネグラス  2017103879   
3            4  201901010101           8            9     キタノコドウ  2017106259   
4            5  201901010101           5            5   ネモフィラブルー  2017104140   
...        ...           ...         ...          ...        ...         ...   
235884      未定  202309050312           6            8  サウンドブライアン  2018103902   
235885      未定  202309050312           7            9   トーホウフランゴ  2020101763   
235886      未定  202309050312           7           10    デルマカンノン  2018103611   
235887      未定  202309050312           8           11     デルマヤクシ  2018100480   
235888      未定  202309050312           8           12   バーンパッション  2017104128   

       AGE  WEIGHT JOCKEY JOCKEY_ID TOR

In [5]:
df = df[['RANKING','HORSEFRAME','HORSENUMBER','WEIGHT','HORSE_WEIGHT','WEIGHT_GAIN_LOSS','ODDS','POPULAR']]
# df = df[['RANKING','HORSEFRAME','HORSENUMBER','WEIGHT','HORSE_WEIGHT','WEIGHT_GAIN_LOSS']]

df = df.replace({'RANKING': {"中止": "999"}})
df = df.replace({'RANKING': {"除外": "999"}})
df = df.replace({'RANKING': {"取消": "999"}})
df = df.replace({'RANKING': {"失格": "999"}})

df = df.astype('float')

print(df)


       RANKING  HORSEFRAME  HORSENUMBER  WEIGHT  HORSE_WEIGHT  \
0          1.0         1.0          1.0    54.0         436.0   
1          2.0         3.0          3.0    54.0         450.0   
2          3.0         5.0          5.0    54.0         412.0   
3          4.0         2.0          2.0    54.0         432.0   
4          5.0         4.0          4.0    54.0         460.0   
...        ...         ...          ...     ...           ...   
47215     11.0         6.0         11.0    57.0         476.0   
47216     12.0         3.0          5.0    49.0         444.0   
47217     13.0         4.0          6.0    52.0         502.0   
47218     14.0         7.0         13.0    54.0         524.0   
47219     15.0         1.0          1.0    51.0         468.0   

       WEIGHT_GAIN_LOSS   ODDS  POPULAR  
0                   0.0    1.2      1.0  
1                  -6.0   11.3      3.0  
2                  -6.0    4.3      2.0  
3                  -2.0   17.2      4.0  
4        

In [ ]:
# 2項分類のため、教師データを２値化する(3着以内なら0,それ以外は1)
df['RANKING'] = np.where(df['RANKING'] <= 3, 0, 1)

# # 'RANKING'列で値が0のものをカウント
# count_zero_ranking = df['RANKING'].eq(0).sum()

# # 結果を表示
# print("値が0の'RANKING'の数:", count_zero_ranking)

df

In [ ]:
# 目的変数と説明変数に分割
x = df.iloc[:,1:] # 説明変数
y = df.iloc[:,0]  # 目的変数

# 訓練用データとテスト用データに分割
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=1)


x_train

# classes = 19

# y_train = to_categorical(y_train, classes)
# y_test = to_categorical(y_test, classes)

,HORSEFRAME,HORSENUMBER,WEIGHT,HORSE_WEIGHT,WEIGHT_GAIN_LOSS,ODDS,POPULAR
25762,8.0,16.0,54.0,424.0,-8.0,353.2,15.0
24175,7.0,7.0,60.0,496.0,6.0,8.4,3.0
32650,6.0,9.0,55.0,480.0,0.0,14.5,4.0
11410,4.0,8.0,55.0,438.0,4.0,70.6,16.0
813,2.0,2.0,54.0,482.0,0.0,13.9,6.0
...,...,...,...,...,...,...,...
43723,3.0,3.0,54.0,466.0,0.0,31.7,9.0
32511,7.0,8.0,54.0,496.0,-4.0,185.6,10.0
5192,5.0,10.0,54.0,418.0,0.0,140.8,16.0
12172,4.0,7.0,54.0,408.0,-4.0,577.1,15.0


In [ ]:
# モデル構造の定義
inputs = keras.layers.Input(shape=(7,)) #7列18行で一塊
x = keras.layers.Flatten()(inputs)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
# predictions = keras.layers.Dense(2, activation='sigmoid')(x)
predictions = keras.layers.Dense(2, activation='softmax')(x)


# 入出力の定義
model = keras.Model(inputs=inputs, outputs=predictions)

# モデルをcompileします
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
print(model.summary())

# 学習します
hist = model.fit(x_train, y_train, epochs=5)

# テストデータの予測精度を計算します
print(model.evaluate(x_test, y_test))

Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 7)]               0         
                                                                 
 flatten_19 (Flatten)        (None, 7)                 0         
                                                                 
 dense_38 (Dense)            (None, 128)               1024      
                                                                 
 dropout_19 (Dropout)        (None, 128)               0         
                                                                 
 dense_39 (Dense)            (None, 2)                 258       
                                                                 
Total params: 1282 (5.01 KB)
Trainable params: 1282 (5.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/5
1033/1033 [

In [ ]:
# モデルの保存
with open('model.pickle', mode='wb') as f:
    pickle.dump(model,f,protocol=2)

NameError: name 'model' is not defined

In [ ]:
# モデルのオープン
with open('model.pickle', mode='rb') as f:
    model = pickle.load(f)

EOFError: Ran out of input

In [ ]:
# DBを基にDFを作成
race_id = "202306050201"
df = convert_instans.convert_df("RESULT_HORSE",race_id)

df = df[['HORSEFRAME','HORSENUMBER','WEIGHT','HORSE_WEIGHT','WEIGHT_GAIN_LOSS','ODDS','POPULAR']]
# df = df[['HORSEFRAME','HORSENUMBER','WEIGHT','HORSE_WEIGHT','WEIGHT_GAIN_LOSS']]

preds = model.predict(df)

print(len(preds))
# print(np.argmax(preds))

NameError: name 'convert_instans' is not defined

In [ ]:
# DBを基にDFを作成
race_id = "202301010102"
df = convert_instans.convert_df("RESULT_HORSE",race_id)

df = df[['HORSEFRAME','HORSENUMBER','WEIGHT','HORSE_WEIGHT','WEIGHT_GAIN_LOSS','ODDS','POPULAR']]
# df = df[['HORSEFRAME','HORSENUMBER','WEIGHT','HORSE_WEIGHT','WEIGHT_GAIN_LOSS']]


# 馬番で並び替え
df = df.sort_values('HORSENUMBER')

# print(df)

# 予測
preds = model.predict(df)


# for pred in preds:
  # number_list.append(pred[0])
  # print(np.argmax(pred))
  
print(preds[8])






NameError: name 'convert_instans' is not defined